In [1]:
import sys
project_path = r"D:/DaiHoc/machinelearning/TLCN/DoAnTotNghiep_chat_bot/"
sys.path.append(project_path)
from source.function.utils_result import RAG
from source.search.utils_search import Qdrant_Utils
from source.rerank.utils_rerank import Rerank_Utils  
from source.model.embedding_model import Sentences_Transformer_Embedding
from source.model.extract_model import Bert_Extract
from source.model.generate_model import Gemini
from source.model.rerank_model import Cohere
from source.data.vectordb.qdrant import Qdrant_Vector
from source.core.config import Settings
from source.generate.generate import Gemini_Generate
from source.extract.utils_extract import Extract_Information
from source.schema.chatbot_querry import ChatbotQuery
setting=Settings()
gemini=Gemini(setting)
print(setting.RERANK)
cohere=Cohere(setting)
print(setting.RERANK)
bert=Bert_Extract(setting)
sentences_transformer_embedding=Sentences_Transformer_Embedding(setting)
qdrant=Qdrant_Vector(setting,sentences_transformer_embedding)
rerank_Utils=Rerank_Utils(cohere)
extract_Utils= Extract_Information(bert)
generate_Utils=Gemini_Generate(gemini,setting)
qdrant_Utils=Qdrant_Utils(qdrant, generate_Utils)
rag=RAG(generate_Utils,extract_Utils,qdrant_Utils,rerank_Utils,setting,sentences_transformer_embedding)
import pandas as pd
from tqdm import tqdm
import google.generativeai as genai 
from sklearn.metrics.pairwise import cosine_similarity
import os

hghaan/rerank_model


tokenizer_config.json:   0%|          | 0.00/1.37k [00:00<?, ?B/s]

c:\Users\hdang\.virtualenvs\TLCN-KU7o-pax\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hdang\.cache\huggingface\hub\models--hghaan--rerank_model. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/22.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

hghaan/rerank_model


## 1.Call API System

In [ ]:
import os
import pandas as pd
from tqdm import tqdm

# Đọc file CSV chứa câu hỏi và câu trả lời đã có (nếu có)
df = pd.read_csv('./data/data_processed/honnhan.csv')

# Nếu chưa có cột 'answer_from_chatbot', thêm cột với giá trị None
if 'answer_from_chatbot' not in df.columns:
    df['answer_from_chatbot'] = None

def sinh_cau_tra_loi_batch(df_batch: pd.DataFrame) -> list[str]:
    """
    Sinh câu trả lời cho mỗi câu hỏi trong batch.
    Trả về danh sách answer tương ứng.
    """
    answers = []
    for question in tqdm(df_batch['question'], desc="Sinh trả lời"):    
        article_doc, _ = rag.get_Article_Content_Results(question)
        answers.append(article_doc)
    return answers

# Chỉ lấy các dòng chưa có câu trả lời (NaN hoặc rỗng)
df_unanswered = df[df['answer_from_chatbot'].isna() | (df['answer_from_chatbot'].str.strip() == '')]

print("==== Bắt đầu sinh trả lời cho các dòng bị thiếu ====")

# Sinh câu trả lời cho toàn bộ các dòng thiếu
df_unanswered['answer_from_chatbot'] = sinh_cau_tra_loi_batch(df_unanswered)

# Cập nhật lại vào df gốc
df.loc[df_unanswered.index, 'answer_from_chatbot'] = df_unanswered['answer_from_chatbot']

# Lưu kết quả cuối cùng
final_output_path = '../Eval_System/data/data_processed/final_response_honnhan.csv'
df.to_csv(final_output_path, index=False)
print(f"✅ Đã cập nhật và lưu toàn bộ vào: {final_output_path}")


==== Bắt đầu sinh trả lời cho các dòng bị thiếu ====


Sinh trả lời batch:   0%|          | 0/100 [00:00<?, ?it/s]

['Câu 1. Tôi có thể kết hôn với cháu ruột của thím mình không?', 'Tôi có được phép kết hôn với người có quan hệ huyết thống là cháu ruột của thím theo quy định pháp luật hôn nhân gia đình không?', 'Quy định pháp luật hiện hành về việc kết hôn giữa những người có quan hệ họ hàng, cụ thể là giữa một người và cháu ruột của thím mình?', 'Điều kiện và hạn chế pháp lý đối với việc kết hôn giữa những người có quan hệ huyết thống trong phạm vi ba đời, ví dụ như trường hợp kết hôn với cháu ruột của thím?']
None
Tìm được
Query 1 có 25 kết quả
None
Tìm được
Query 2 có 25 kết quả
None
Tìm được
Query 3 có 25 kết quả
None
Tìm được
Query 4 có 25 kết quả
Tổng số kết quả tìm được: 100
Đã thực hiện xong retrival
Số document retrival được 4
Số document khi xoá trùng 39
Đang sử dụng mô hình fine-tune để xếp hạng lại tài liệu.
An error occurred: 'Cohere' object has no attribute 'predict'
Số document sau khi qua rerank: 0


Sinh trả lời batch:   1%|          | 1/100 [00:23<38:01, 23.04s/it]

['Câu 2. Anh J và chị O học tiểu học cùng nhau, anh J theo bố mẹ sang định cư tại Đan Mạch. Khi về thăm quê, anh J có gặp lại chị O, từ đó cả hai nối lại tình bạn. Sau một thời gian trao đổi, liên hệ với nhau qua điện thoại, mạng xã hội, chị O tỏ ý muốn sang định cư tại Đan Mạch và nhờ anh J giúp đỡ bằng cách đồng ý kết hôn với chị. Hai bên sẽ ly hôn sau khi chị O được nhập quốc tịch và đã sang cư trú tại Đan Mạch. Trưởng hợp này pháp luật có nghiêm cấm không và nếu J và O vẫn thực hiện thì xử lý như thế nào?', 'Hành vi kết hôn giả tạo để nhập quốc tịch Đan Mạch có bị pháp luật Việt Nam cấm không và chế tài xử lý nếu vi phạm?', 'Hôn nhân có yếu tố gian dối nhằm mục đích nhập tịch nước ngoài, cụ thể là Đan Mạch, có vi phạm quy định pháp luật Việt Nam không và hậu quả pháp lý?', 'Pháp luật Việt Nam quy định như thế nào về việc xử lý hành vi lợi dụng kết hôn để nhập quốc tịch nước ngoài (Đan Mạch) rồi ly hôn?']
None
Tìm được
Query 1 có 25 kết quả
None
Tìm được
Query 2 có 25 kết quả
None
T

Sinh trả lời batch:   2%|▏         | 2/100 [00:44<36:28, 22.33s/it]


KeyboardInterrupt: 

### 2.Eval System

> Configure

In [ ]:
setting=Settings()

In [ ]:
genai.configure(api_key=gemini.key_manager.get_next_key())

> Get embedding

In [ ]:
def get_gemini_embedding(text: str,
                         model_name: str = "models/text-embedding-004") -> list[float]:
    res = genai.embed_content(
        model=model_name,
        content=text
    )
    return res["embedding"]

In [ ]:
df = pd.read_csv('./data/data_processed/data_final_response_100_199.csv')
# Lấy embedding cho Answer và answer_from_chatbot
df['answer_embedding'] = df['answer'].apply(get_gemini_embedding)
df['answer_from_chatbot_embedding'] = df['answer_from_chatbot'].apply(get_gemini_embedding)

# Tính cosine similarity giữa Answer và answer_from_chatbot
df['cosine_similarity'] = df.apply(
    lambda row: cosine_similarity(
        [row['answer_embedding']],
        [row['answer_from_chatbot_embedding']]
    )[0][0],
    axis=1
)
# Tính trung bình của cosine similarity
average_cosine_similarity = df['cosine_similarity'].mean()

print(f"Trung bình cosine similarity giữa Answer và answer_from_chatbot: {average_cosine_similarity:.4f}")

In [ ]:
from sentence_transformers import CrossEncoder

def rerank_model(query, documents):
    if len(documents) > 50:
        raise ValueError("Số lượng documents không được vượt quá 50.")

    model_repo = "hghaan/rerank_model"
    model = CrossEncoder(model_repo, trust_remote_code=True)

    pairs = [(query, doc) for doc in documents]
    scores = model.predict(pairs)
    ranked = sorted(zip(scores, documents), key=lambda x: x[0], reverse=True)

    return ranked[:5]

c:\Users\hdang\.virtualenvs\TLCN-KU7o-pax\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
